In [12]:
import os
import h5py

import cv2
import numpy

import time
import math

In [13]:
DATA_FORMAT = ".png"
DATAPATH_TRAIN = "./imageTrain/"
DATAPATH_VALIDATE = "./imageValidate/"

RANDOM_CROP = 30
SIZE_PATCH = 32
SIZE_CONV = 6
SIZE_LABEL = (SIZE_PATCH - SIZE_CONV * 2)

SCALE = 2
INTERPOLATION = cv2.INTER_CUBIC

FILENAME_TRAIN = "dataTrain_" + str(SCALE) + ".h5"
FILENAME_VALIDATE = "dataValidate_" + str(SCALE) + ".h5"

In [14]:
def parseData(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = numpy.zeros((nums * RANDOM_CROP, 1, SIZE_PATCH, SIZE_PATCH), dtype=numpy.double)
    label = numpy.zeros((nums * RANDOM_CROP, 1, SIZE_LABEL, SIZE_LABEL), dtype=numpy.double)

    for i in range(2):
        if DATA_FORMAT in names[i]:
            name = _path + names[i]

            hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
            #hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
            shape = hr_img.shape

            hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
            hr_img = hr_img[:, :, 0]

            lr_img = cv2.resize(hr_img, (int(shape[1] / SCALE), int(shape[0] / SCALE)), INTERPOLATION)
            lr_img = cv2.resize(lr_img, (shape[1], shape[0]), INTERPOLATION)

            Points_x = numpy.random.randint(0, min(shape[0], shape[1]) - SIZE_PATCH, RANDOM_CROP)
            Points_y = numpy.random.randint(0, min(shape[0], shape[1]) - SIZE_PATCH, RANDOM_CROP)

            for j in range(RANDOM_CROP):
                lr_patch = lr_img[Points_x[j]: Points_x[j] + SIZE_PATCH, Points_y[j]: Points_y[j] + SIZE_PATCH]
                hr_patch = hr_img[Points_x[j]: Points_x[j] + SIZE_PATCH, Points_y[j]: Points_y[j] + SIZE_PATCH]

                lr_patch = lr_patch.astype(float) / 255.
                hr_patch = hr_patch.astype(float) / 255.

                data[i * RANDOM_CROP + j, 0, :, :] = lr_patch
                label[i * RANDOM_CROP + j, 0, :, :] = hr_patch[SIZE_CONV: -SIZE_CONV, SIZE_CONV: -SIZE_CONV]

                #plt.figure(figsize=(16,16))
                #plt.subplot(1,2,1)
                #plt.title('lr')
                #plt.imshow(lr_patch)
                #plt.subplot(1,2,2)
                #plt.title('hr')
                #plt.imshow(hr_patch)
    
    return data, label

In [15]:
BLOCK_STEP = 16
BLOCK_SIZE = 32

In [16]:
def parseCropData(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = []
    label = []

    for i in range(nums):
        if DATA_FORMAT in names[i]:
            name = _path + names[i]
            hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
            hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
            hr_img = hr_img[:, :, 0]
            shape = hr_img.shape

            lr_img = cv2.resize(hr_img, (int(shape[1] / SCALE), int(shape[0] / SCALE)))
            lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

            width_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
            height_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
            for k in range(int(width_num)):
                for j in range(int(height_num)):
                    x = k * BLOCK_STEP
                    y = j * BLOCK_STEP
                    hr_patch = hr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                    lr_patch = lr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]

                    lr_patch = lr_patch.astype(float) / 255.
                    hr_patch = hr_patch.astype(float) / 255.

                    lr = numpy.zeros((1, SIZE_PATCH, SIZE_PATCH), dtype=numpy.double)
                    hr = numpy.zeros((1, SIZE_LABEL, SIZE_LABEL), dtype=numpy.double)

                    lr[0, :, :] = lr_patch
                    hr[0, :, :] = hr_patch[SIZE_CONV: -SIZE_CONV, SIZE_CONV: -SIZE_CONV]

                    data.append(lr)
                    label.append(hr)

    data = numpy.array(data, dtype=float)
    label = numpy.array(label, dtype=float)
    return data, label

In [17]:
def h5DataWrite(data, labels, output_filename):
    x = data.astype(numpy.float32)
    y = labels.astype(numpy.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)

In [18]:
tick1 = cv2.getTickCount()

In [19]:
data, label = parseCropData(DATAPATH_TRAIN)
h5DataWrite(data, label, FILENAME_TRAIN)
print(FILENAME_TRAIN + " generated")
data, label = parseData(DATAPATH_VALIDATE)
h5DataWrite(data, label, FILENAME_VALIDATE)
print(FILENAME_VALIDATE + " generated")

dataTrain_2.h5 generated
dataValidate_2.h5 generated


In [20]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())
print("processed time: " + str(tick) + " ms")

processed time: 2129 ms
